# 세팅

In [ ]:
!pip install psychrochart

In [ ]:
import pandas as pd
import numpy as np
from psychrochart import load_config, PsychroChart
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

## 결로 여부 판단을 위한 코드

In [ ]:
import math as m

# All functions expect base SI units for any arguments given
# DBT   - Dry bulb temperature          - Kelvins, K
# DPT   - Dew point temperature         - Kelvins, K
# H     - Specific enthalpy             - kiloJoules per kilogram, kJ/kg
# P     - Atmospheric pressure          - Pascals, Pa
# Pw    - Water vapor partial pressure  - Pascals, Pa
# RH    - Relative humidity             - Decimal (i.e. not a percentage)
# V     - Specific volume               - Cubic meters per kilogram, m^3/kg
# W     - Humidity ratio                - kilograms per kilograms, kg/kg
# WBT   - Wet bulb temperature          - Kelvins, K

# Minimum dry bulb temperature
Min_DBT=273.15
# Maximum dry bulb temperature
Max_DBT=473.15
# Convergence tolerance
TOL=0.0005

def __DBT_H_RH_P(H, RH, P):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=__W_DBT_RH_P(DBTa, RH, P)-__W_DBT_H(DBTa, H)
        y=__W_DBT_RH_P(DBT, RH, P)-__W_DBT_H(DBT, H)
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

def __DBT_H_V_P(H, V, P):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=__W_DBT_V_P(DBTa, V, P)-__W_DBT_H(DBTa, H)
        y=__W_DBT_V_P(DBT, V, P)-__W_DBT_H(DBT, H)
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

def __DBT_H_W(H, W):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=W-__W_DBT_H(DBTa, H)
        y=W-__W_DBT_H(DBT, H)
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

def __DBT_H_WBT_P(H, WBT, P):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=__W_DBT_WBT_P(DBTa, WBT, P)-__W_DBT_H(DBTa, H)
        y=__W_DBT_WBT_P(DBT, WBT, P)-__W_DBT_H(DBT, H)
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

def __DBT_RH_V_P(RH, V, P):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=__W_DBT_RH_P(DBTa, RH, P)-__W_DBT_V_P(DBTa, V, P)
        y=__W_DBT_RH_P(DBT, RH, P)-__W_DBT_V_P(DBT, V, P)
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

def __DBT_RH_W_P(RH, W, P):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=__W_DBT_RH_P(DBTa, RH, P)-W
        y=__W_DBT_RH_P(DBT, RH, P)-W
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

def __DBT_RH_WBT_P(RH, WBT, P):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=__W_DBT_WBT_P(DBTa, WBT, P)-__W_DBT_RH_P(DBTa, RH, P)
        y=__W_DBT_WBT_P(DBT, WBT, P)-__W_DBT_RH_P(DBT, RH, P)
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

def __DBT_V_W_P(V, W, P):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=W-__W_DBT_V_P(DBTa, V, P)
        y=W-__W_DBT_V_P(DBT, V, P)
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

def __DBT_V_WBT_P(V, WBT, P):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=__W_DBT_WBT_P(DBTa, WBT, P)-__W_DBT_V_P(DBTa, V, P)
        y=__W_DBT_WBT_P(DBT, WBT, P)-__W_DBT_V_P(DBT, V, P)
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

def __DBT_W_WBT_P(W, WBT, P):
    [DBTa, DBTb]=[Min_DBT, Max_DBT]
    DBT=(DBTa+DBTb)/2
    while DBTb-DBTa>TOL:
        ya=__W_DBT_WBT_P(DBTa, WBT, P)-W
        y=__W_DBT_WBT_P(DBT, WBT, P)-W
        if __is_positive(y)==__is_positive(ya):
            DBTa=DBT
        else:
            DBTb=DBT
        DBT=(DBTa+DBTb)/2
    return DBT

# ASHRAE 2009 Chapter 1 Equation 39
def __DPT_Pw(Pw):
    Pw=Pw/1000
    C14=6.54
    C15=14.529
    C16=0.7389
    C17=0.09486
    C18=0.4569
    a=m.log(Pw)
    return (C14+C15*a+C16*a**2+C17*a**3+C18*Pw**0.1984)+273.15

# ASHRAE 2009 Chapter 1 Equation 32
def __H_DBT_W(DBT, W):
    if __valid_DBT(DBT):
        DBT=DBT-273.15
        return 1.006*DBT+W*(2501+1.86*DBT)

def __is_positive(x):
    if x>0:
        return True
    else:
        return False

# ASHRAE 2009 Chapter 1 Equation 22
def __Pw_W_P(W, P):
    return W*P/(W+0.621945)

# ASHRAE 2009 Chapter 1 Equation 6
def __Pws(DBT):
    if __valid_DBT(DBT):
        C8=-5.8002206*10**3
        C9=1.3914993
        C10=-4.8640239*10**-2
        C11=4.1764768*10**-5
        C12=-1.4452093*10**-8
        C13=6.5459673
        return m.exp(C8/DBT+C9+C10*DBT+C11*DBT**2+C12*DBT**3+C13*m.log(DBT))

def state(prop1, prop1val, prop2, prop2val,P):
    if prop1==prop2:
        print("Properties must be independent.")
        return
    prop=["DBT","WBT","RH","W","V","H"]
    if prop1 not in prop or prop2 not in prop:
        print("Valid property must be given.")
        return
    prop1i=prop.index(prop1)
    prop2i=prop.index(prop2)
    if prop1i<prop2i:
        cd1=prop1
        cd1val=prop1val
        cd2=prop2
        cd2val=prop2val
    else:
        cd1=prop2
        cd1val=prop2val
        cd2=prop1
        cd2val=prop1val
    if cd1=="DBT":
        DBT=cd1val
        if cd2=="WBT":
            WBT=cd2val
            W=__W_DBT_WBT_P(DBT, WBT, P)
            H=__H_DBT_W(DBT, W)
            RH=__RH_DBT_W_P(DBT, W, P)
            V=__V_DBT_W_P(DBT, W, P)
        elif cd2=="RH":
            RH=cd2val
            W=__W_DBT_RH_P(DBT, RH, P)
            H=__H_DBT_W(DBT, W)
            V=__V_DBT_W_P(DBT, W, P)
            WBT=__WBT_DBT_W_P(DBT, W, P)
        elif cd2=="W":
            W=cd2val
            H=__H_DBT_W(DBT, W)
            RH=__RH_DBT_W_P(DBT, W, P)
            V=__V_DBT_W_P(DBT, W, P)
            WBT=__WBT_DBT_W_P(DBT, W, P)
        elif cd2=="V":
            V=cd2val
            W=__W_DBT_V_P(DBT, V, P)
            H=__H_DBT_W(DBT, W)
            RH=__RH_DBT_W_P(DBT, W, P)
            WBT=__WBT_DBT_W_P(DBT, W, P)
        elif cd2=="H":
            H=cd2val
            W=__W_DBT_H(DBT, H)
            RH=__RH_DBT_W_P(DBT, W, P)
            V=__V_DBT_W_P(DBT, W, P)
            WBT=__WBT_DBT_W_P(DBT, W, P)
    elif cd1=="WBT":
        WBT=cd1val
        if cd2=="RH":
            RH=cd2val
            DBT=__DBT_RH_WBT_P(RH, WBT, P)
            W=__W_DBT_RH_P(DBT, RH, P)
            H=__H_DBT_W(DBT, W)
            V=__V_DBT_W_P(DBT, W, P)
        elif cd2=="W":
            W=cd2val
            DBT=__DBT_W_WBT_P(W, WBT, P)
            H=__H_DBT_W(DBT, W)
            RH=__RH_DBT_W_P(DBT, W, P)
            V=__V_DBT_W_P(DBT, W, P)
        elif cd2=="V":
            V=cd2val
            DBT=__DBT_V_WBT_P(V, WBT, P)
            W=__W_DBT_V_P(DBT, V, P)
            H=__H_DBT_W(DBT, W)
            RH=__RH_DBT_W_P(DBT, W, P)
        elif cd2=="H":
            H=cd2val
            DBT=__DBT_H_WBT_P(H, WBT, P)
            W=__W_DBT_H(DBT, H)
            RH=__RH_DBT_W_P(DBT, W, P)
            V=__V_DBT_W_P(DBT, W, P)
    elif cd1=="RH":
        RH=cd1val
        if cd2=="W":
            W=cd2val
            DBT=__DBT_RH_W_P(RH, W, P)
            H=__H_DBT_W(DBT, W)
            V=__V_DBT_W_P(DBT, W, P)
            WBT=__WBT_DBT_W_P(DBT, W, P)
        elif cd2=="V":
            V=cd2val
            DBT=__DBT_RH_V_P(RH, V, P)
            W=__W_DBT_RH_P(DBT, RH, P)
            H=__H_DBT_W(DBT, W)
            WBT=__WBT_DBT_W_P(DBT, W, P)
        elif cd2=="H":
            H=cd2val
            DBT=__DBT_H_RH_P(H, RH, P)
            W=__W_DBT_RH_P(DBT, RH, P)
            V=__V_DBT_W_P(DBT, W, P)
            WBT=__WBT_DBT_W_P(DBT, W, P)
    elif cd1=="W":
        W=cd1val
        if cd2=="V":
            V=cd2val
            DBT=__DBT_V_W_P(V, W, P)
            H=__H_DBT_W(DBT, W)
            RH=__RH_DBT_W_P(DBT, W, P)
            WBT=__WBT_DBT_W_P(DBT, W, P)
        elif cd2=="H":
            H=cd2val
            DBT=__DBT_H_W(H, W)
            RH=__RH_DBT_W_P(DBT, W, P)
            V=__V_DBT_W_P(DBT, W, P)
            WBT=__WBT_DBT_W_P(DBT, W, P)
    elif cd1=="V":
        V=cd1val
        H=cd2val
        DBT=__DBT_H_V_P(H, V, P)
        W=__W_DBT_V_P(DBT, V, P)
        RH=__RH_DBT_W_P(DBT, W, P)
        WBT=__WBT_DBT_W_P(DBT, W, P)
    return [DBT, H, RH, V, W, WBT]

# ASHRAE 2009 Chapter 1 Equation 22 and Equation 24
def __RH_DBT_W_P(DBT, W, P):
    if __valid_DBT(DBT):
        return W*P/((0.621945+W)*__Pws(DBT))

# ASHRAE 2009 Chapter 1 Equation 28
def __V_DBT_W_P(DBT, W, P):
    if __valid_DBT(DBT):
        return 287.042*DBT*(1+1.607858*W)/P

# ASHRAE 2009 Chapter 1 Equation 32
def __W_DBT_H(DBT, H):
    if __valid_DBT(DBT):
        DBT=DBT-273.15
        return (H-1.006*DBT)/(2501+1.86*DBT)

# ASHRAE 2009 Chapter 1 Equation 22 and Equation 24
def __W_DBT_RH_P(DBT, RH, P):
    if __valid_DBT(DBT):
        Pw=RH*__Pws(DBT)
        return 0.621945*Pw/(P-Pw)

# ASHRAE 2009 Chapter 1 Equation 28
def __W_DBT_V_P(DBT, V, P):
    if __valid_DBT(DBT):
        return (P*V-287.042*DBT)/(1.607858*287.042*DBT)

# ASHRAE 2009 Chapter 1 Equation 35
def __W_DBT_WBT_P(DBT, WBT, P):
    if __valid_DBT(DBT):
        DBT=DBT-273.15
        WBT=WBT-273.15
        return ((2501-2.326*WBT)*__W_DBT_RH_P(WBT+273.15,1,P)-1.006*(DBT-WBT))/\
               (2501+1.86*DBT-4.186*WBT)

# ASHRAE 2009 Chapter 1 Equation 35
def __WBT_DBT_W_P(DBT, W, P):
    if __valid_DBT(DBT):
        WBTa=__DPT_Pw(__Pw_W_P(W, P))
        WBTb=DBT
        WBT=(WBTa+WBTb)/2
        while WBTb-WBTa>TOL:
            Ws=__W_DBT_WBT_P(DBT, WBT, P)
            if W>Ws:
                WBTa=WBT
            else:
                WBTb=WBT
            WBT=(WBTa+WBTb)/2
        return WBT

def __valid_DBT(DBT):
    if Min_DBT<=DBT<=Max_DBT:
        return True
    else:
        return False

# 계산식 구성

In [ ]:
# Dry bulb temperature (DBT)
# Specific enthalpy (H)
# Relative humidity (RH)
# Specific volume (V)
# Humidity ratio (W)
# Wet bulb temperature (WBT)
# 건구, H, 상대습도, V, 절대습도, 습구
Kel = 273.15

In [ ]:
def rr (v):
  v = round(v,2)
  return v

In [ ]:
def KorC (t):
  if t<=100:
    # t는 C
    t = t+Kel
    return t
  else:
    # t는 K
    return t

## 실행 확인을 위한 샘플 설정

In [ ]:
# 구간 설정에 따른 온습도 설정값 반환
def section (i):
  [t1,t2,opt1,opt2,oph1,oph2] = [0,0,0,0,0,0]

  # if i == 1:
  #   t1 = 400
  #   t2 = 600
  #   opt1 = 18.0
  #   opt2 = 21.0
  #   oph1 = 60.0
  #   oph2 = 80.0
  #   return [t1,t2,opt1,opt2,oph1,oph2]

  # if i == 2:
  #   t1 = 0600
  #   t2 = 1800
  #   opt1 = 18.0
  #   opt2 = 21.0
  #   oph1 = 60.0
  #   oph2 = 80.0
  #   return [t1,t2,opt1,opt2,oph1,oph2]

  if i == 3:
    t1 = 1800
    t2 = 2200
    opt1 = 15.0 #생육한계저온
    opt2 = 25.0 #생육적온
    opt3 = 30.0 #생육한계고온
    oph1 = 60.0 #저습한계
    oph2 = 80.0 #고습한계
    return [t1,t2,opt1,opt2,opt3,oph1,oph2]

  # if i == 4:
  #   t1 = 2200
  #   t2 = 2359
  #   opt1 = 18.0
  #   opt2 = 21.0
  #   oph1 = 60.0
  #   oph2 = 80.0
  #   return [t1,t2,opt1,opt2,oph1,oph2]

In [ ]:
# 건구,상대습도에 따른 값 반환
def psycalTRH (t, h):
  a = KorC(t)
  b = rr(h/100)
  outlist1 = state("DBT",a,"RH",b,101325)
  return outlist1

In [ ]:
# 건구, 절대습도에 따른 값 반환
def psycalTW (t, w):
  a = KorC(t)
  outlist2 = state("DBT",a,"W",w,101325)
  return outlist2

In [ ]:
# 건구 상대습도
def cal_temp_hum (t, h, s):
  temp = t
  rhum = h
  ss = section(s)
  # 작동 불가 상황
  # 한계저온 이하
  if temp<=ss[2]:
    print('온도가 너무 낮습니다.')
  # 다습  
  elif rhum >=ss[6]:
    print('습도가 너무 높습니다.')

  # 작동 가능 상황
  elif (temp>ss[2]) & (rhum<ss[6]):
    ex1 = psycalTRH(temp, rhum)
    hr1 = ex1[4]
    settemp = temp
    p1 = temp
    # 누적적온 계산 값 추가(4단계 호흡감소기에만 적용)

    # 적정 온도 + 적정 습도
    if (temp<ss[3]) & (ss[5]<rhum):
      while (ss[2]<settemp):
        settemp = settemp-0.01
        # settemp와 절대습도를 기준으로 상대습도 값 계산
        ex2 = psycalTW(settemp, hr1)
        rhum2 = ex2[2]
        rhum2 = rhum2*100

        if (ss[2]<=settemp) & (ss[5]<=rhum2) & (rhum2<ss[6]):
          st = settemp
          rh = rhum2
          continue
        # and 설정으로 온도가 벗어나거나 습도가 벗어나면 반복 종료  
        else:
          st = rr(st)
          rh = rr(rh)
          cal = [temp,rhum,st,rh]
          return cal, ss

    # 한계고온 이상
    elif ss[4]<=temp:
      while ss[2]<=settemp:
        settemp = settemp-0.01
        ex2 = psycalTW(settemp, hr1)
        rhum2 = ex2[2]
        rhum2 = rhum2*100

        if (ss[2]<=settemp) & (rhum2<ss[6]):
          st = settemp
          rh = rhum2
          continue
        else:
          st = rr(st)
          rh = rr(rh)
          if st>ss[4]:
            print('냉방 필요')
          cal = [temp,rhum,st,rh]
          return cal, ss
    
    # 건조
    elif ss[5]>=rhum:
      while ss[2]<=settemp:
        settemp = settemp-0.01
        ex2 = psycalTW(settemp, hr1)
        rhum2 = ex2[2]
        rhum2 = rhum2*100

        if (ss[2]<=settemp) & (rhum2<ss[6]):
          st = settemp
          rh = rhum2
          continue
        else:
          st = rr(st)
          rh = rr(rh)
          if rh<ss[5]:
            print('습 보충 필요')
          cal = [temp,rhum,st,rh]
          return cal, ss

In [ ]:
cal_set = cal_temp_hum(20,70,3)
print(cal_set[0])

[20, 70, 17.87, 79.96]


# 계산 및 시각화

In [ ]:
plt.rcParams['figure.figsize'] = [3, 3] # [width, height] (inches)

In [ ]:
# Get a preconfigured chart
chart = PsychroChart("minimal")

# Append zones:
zones_conf = {
    "zones":[{
            "zone_type": "dbt-rh",
            "style": {"edgecolor": [1.0, 0.749, 0.0, 0.8],
                      "facecolor": [1.0, 0.749, 0.0, 0.2],
                      "linewidth": 2,
                      "linestyle": "--"},
            "points_x": [cal_set[1][2], cal_set[1][4]],
            "points_y": [cal_set[1][5], cal_set[1][6]],
            "label": "Preferred section"
          }]}

chart.append_zones(zones_conf)

# Plot the chart
ax = chart.plot()

# 각 포인터 설정
points = {'now': {'label': 'now (%s,%s)'% (cal_set[0][0],cal_set[0][1]),
                       'style': {'color': [0.855, 0.004, 0.278, 0.8],
                                 'marker': 's', 'markersize': 10},
                       'xy': (cal_set[0][0],cal_set[0][1])},
          'set': {'label': 'set (%s,%s)'% (cal_set[0][2],cal_set[0][3]),
                       'style': {'color': [0.592, 0.745, 0.051, 0.9],
                                 'marker': 'D', 'markersize': 20},
                       'xy': (cal_set[0][2],cal_set[0][3])}}



# # 박스 설정
# box = {'boxstyle': 'square','ec': (0.5, 0.5, 1.0),'fc': (0.8, 0.8, 1.0),'linestyle': '--'}
# # 폰트 설정
# font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}

# # ax.text(cal_set[0],cal_set[1],cal_set[0],fontsize=10,bbox=box)
# # bbox_position_size = 'bottom'

# # 포인터 값 표시
# ax.text(cal_set[0], cal_set[1], cal_set[0], fontdict=font, bbox=box)


# 포인터 사이 연결
connectors = [{'start': 'now',
               'end': 'set',
               'label': 'process',
               'style': {'color': [0.573, 0.106, 0.318, 0.7],
                         "linewidth": 3, "linestyle": "-."}}]
chart.plot_points_dbt_rh(points, connectors)

# Add a legend
chart.plot_legend(markerscale=.7, frameon=False, fontsize=10, labelspacing=1.2)
ax.get_figure()